# TP1

## Parte 1  (_/white-patch_ y _/coord-cromáticas_)

#### 1) Implementar el algoritmo de pasaje a coordenadas cromáticas para librarnos de las variaciones de contraste


Al leer una imagen con OpenCV, esta es representada por defecto en el espacio de color de BGR. Esta representación resulta muy sensible al cambio de contraste ya que el brillo está caracterizado de forma marginal en cada una de sus coordenada (B, G, y R). Según lo solicitado en el enunciado, debemos tomar la imagen y representarla en otro espacio de color, cuyas componentes sean menos sensibles a la iluminación de la imagen (por ejemplo, con HSV, donde el cambio de brillo sólo afecta la componente de Value).

_(Nota: OpenCV ya implementa una función que realiza esta transformación, llamada cvtColor, pero haremos nuestra propia implementación para satisfacer el enunciado)._



In [13]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

# Intentemos hacer la conversión de manera vectorizada para que sea mas eficiente:
def bgr_to_HSV(img):
    
    # Normalizamos todos los valores a un rango de 0 a 1 dividiendo por 255
    img /= 255
    
    # Nos quedamos con la matriz de cada canal por separado
    b, g, r = cv.split(img)

    
    
    # Según las formulas del modelo HSV, necesitamos el máximo y 
    # mínimo de las tres componentes, y la diferencia:
    max = np.max(img, axis=-1)
    min = np.min(img, axis=-1)
    diff = max - min

    # Inicializamos los canales H, S y V:
    h = np.zeros_like(max)
    s = np.zeros_like(max)
    
    # El Value es simplemente el color máximo del pixel, así que lo inicializamos así
    v = max

    # Calculamos la saturación cuando la diferencia entre maximo y minimo
    # no es nula, si no dejamos la saturacion del pixel en 0
    s[diff > 0] = diff[diff > 0] / max[diff > 0]

    # Como la formula del tono (Hue) cambia según que color es el máximo en el pixel,
    # creamos máscaras y utilizamos cada formula por separado

    # cuando R es el máximo
    mask = (max == r) & (diff > 0)
    h[mask] = (((g[mask] - b[mask]) / diff[mask]) % 6) * 60
    
    # cuando G es el máximo
    mask = (max == g) & (diff > 0)
    h[mask] = (((b[mask] - r[mask]) / diff[mask]) + 2) * 60
    
    # cuando B es el máximo
    mask = (max == b) & (diff > 0)
    h[mask] = (((r[mask] - g[mask]) / diff[mask]) + 4) * 60
    
    # Para comparar los resultados con la función de OpenCV, trasladamos los rangos de
    # cada componente (H entre 0 y 179, S y V entre 0 y 255)
    h /= 2
    s = s * 255
    v = v * 255
    
    # Devolvemos la matriz con los componentes calculados
    return np.stack([h, s, v], axis=-1)

img = cv.imread('coord_cromaticas/CoordCrom_1.png')
img = img.astype(float)

img_hsv = bgr_to_HSV(img)
#img_hsv = cv.cvtColor(img,cv.COLOR_BGR2HSV)
print(img_hsv)
#print(img)

[[[141.42857143   8.22580645 217.        ]
  [145.           7.08333333 216.        ]
  [145.71428571   8.22580645 217.        ]
  ...
  [  9.31034483  35.38277512 209.        ]
  [  9.31034483  35.89805825 206.        ]
  [  7.74193548  38.1884058  207.        ]]

 [[150.           8.26388889 216.        ]
  [150.           8.26388889 216.        ]
  [150.           8.26388889 216.        ]
  ...
  [  9.31034483  36.25       204.        ]
  [  9.31034483  35.72463768 207.        ]
  [  9.31034483  36.07317073 205.        ]]

 [[150.           7.08333333 216.        ]
  [145.           7.05069124 217.        ]
  [145.           7.05069124 217.        ]
  ...
  [ 10.34482759  36.07317073 205.        ]
  [ 10.          37.13592233 206.        ]
  [ 11.61290323  38.75       204.        ]]

 ...

 [[ 11.50684932 105.76704545 176.        ]
  [ 11.91780822 105.16949153 177.        ]
  [ 11.91780822 103.41666667 180.        ]
  ...
  [ 11.91780822 124.93288591 149.        ]
  [ 11.91780822 12